In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import os

eth = pd.read_parquet('addr/address_map_eth.parquet')
trx = pd.read_parquet('addr/address_map_usdt_tron.parquet')

In [4]:
addr = pd.concat([
    v.rename(columns={'replace_name':'name'})
    for v in ([eth, trx] + [
        pd.read_csv(f'addr/{f}') for f in [
            'non_exchange_address.csv', 
            'exchange_address2.csv',
            'address_kor_exc.csv',
        ]
    ])
])
addr.name = addr.name.str.lower()
addr.name = addr.name.str.replace('_deposit', '')
addr.name = addr.name.str.replace('defi-', '')
addr.exchange_name = addr.exchange_name.str.lower()


for v in ['isContract', 'isWhale']:
    addr[v] = False
for name, elements in {
   'network': ['eth', 'trx']
}.items():
    addr[name] = None
    addr[name] = addr[name].astype(
        CategoricalDtype(categories=elements, ordered=False)
    )

for name, elements in {
    "serviceType": ['CeFi', 'DeFi', 'Gov', 'Other'],
    'sub1Service': [
        'exc', 'lend', 'custody', 'fund', 
        'bridge', 'agg', 'yield', 'oracle', 'LP', 'otc',
        'dao', 'susp', 'MEV', 'treasury', 'chain',  'etc',
    ],
    'walletPurpose': ['Staking', 'Infra', 'Vault', 'Gov', 'Other'],
    'sub1Wallet':[
        'soloStaker', 'stakingPool', 
        'multisig', 'mint', 'validator', 
        'hot', 'cold', 'liqPool', 'deposit',
        'voter', 'delegate', 'delegator',
        'proxy', 'router', 'resolver', 'marketMaker',
    ],
}.items():
    addr[name] = None
    addr[name] = addr[name].astype(
        CategoricalDtype(categories=elements, ordered=False)
    )

is_contain_in_name = lambda x: addr.name.str.contains(x, regex=True, na=False)
is_end = lambda x: addr.name.str.endswith(x, na=False)
    
addr

,address,name,exchange_name,isContract,isWhale,network,serviceType,sub1Service,walletPurpose,sub1Wallet
0,0x983873529f95132bd1812a3b52c98fb271d2f679,ascendex,NaN,False,False,NaN,NaN,NaN,NaN,NaN
1,0x19184ab45c40c2920b0e0e31413b9434abd243ed,binance,NaN,False,False,NaN,NaN,NaN,NaN,NaN
2,0x25681ab599b4e2ceea31f8b498052c53fc2d74db,binance,NaN,False,False,NaN,NaN,NaN,NaN,NaN
3,0x2859e4544c4bb03966803b044a93563bd2d0dd4d,binance,NaN,False,False,NaN,NaN,NaN,NaN,NaN
4,0x2bc4eb013ddee29d37920938b96d353171289b7c,binance,NaN,False,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
14965,0xd388889dea826e64757abfeeda952ca2a6c6e6c2,bithumb,NaN,False,False,NaN,NaN,NaN,NaN,NaN
14966,0x6720e2e748baec9e1f2efef4745fd86ec2d101b3,bithumb,NaN,False,False,NaN,NaN,NaN,NaN,NaN
14967,0xb67750b1745349fbd6dd6477fd5a05070d9e3e0c,bithumb,NaN,False,False,NaN,NaN,NaN,NaN,NaN
14968,0xb46a19d227bdb8a48fae93e1e9abd897f7c0fee1,bithumb,NaN,False,False,NaN,NaN,NaN,NaN,NaN


In [5]:

    
addr.loc[addr.exchange_name.isna(), 'serviceType'] = "CeFi"
addr.loc[addr.exchange_name.isna(), 'sub1Service'] = "exc"
addr.loc[~addr.exchange_name.isna(), 'serviceType'] = "DeFi"
addr.loc[addr.name.isna(), 'name'] = addr.loc[addr.name.isna(), 'exchange_name']

addr.loc[addr.address.str.contains(
    r'(null|0000000000000000000000000000)', regex=True, na=False), 'name'] = 'trash'

for k, v in {
    'treasury': [r'(circle|tether)'],
    'etc': [r'(opensea|metamask)', r'(null|0000000000000000000000000000)']
}.items():
    for vv in v:        
        addr.loc[is_contain_in_name(vv), 'serviceType'] = 'Other'
        addr.loc[is_contain_in_name(vv), 'sub1Service'] = k

addr.loc[is_end('_network'), 'serviceType'] = 'Other'
addr.loc[is_end('_network'), 'sub1Service'] = 'chain'
addr.loc[addr.name.isin(['base', 'optimism', 'polygon']), 'serviceType'] = 'Other'
addr.loc[addr.name.isin(['base', 'optimism', 'polygon']), 'sub1Service'] = 'chain'


addr.loc[is_end('custody'), 'serviceType'] = 'DeFi'
addr.loc[is_end('custody'), 'sub1Service'] = 'custody'
addr.loc[addr.name=='bridgers', 'serviceType'] = 'DeFi'
addr.loc[addr.name=='bridgers', 'sub1Service'] = 'bridge'



addr = addr.drop(columns=['exchange_name'])

addr.loc[addr.name=="bistamp", 'name'] = 'bitstamp'
addr.loc[addr.name=="smartexchangerouter", 'name'] = 'suniosmartexchangerouter'
addr.loc[addr.name=="btseexchange", 'name'] = 'btse'
addr.loc[addr.name=="sky(makerdao)", 'name'] = 'makerdao'
addr.loc[addr.sub1Service=='chain', 'name'] = addr.loc[addr.sub1Service=='chain', 'name'].str.replace('_network', '')
addr.loc[addr.sub1Service=='custody', 'name'] = addr.loc[addr.sub1Service=='custody', 'name'].str.replace('custody', '')


addr.loc[addr.address.str[:2]=='0x', 'network'] = 'eth'
addr.loc[addr.address.str[0]=='T', 'network'] = 'trx'
addr.address = addr.address.str.lower()
addr = addr.drop_duplicates()

addr

C:\Users\Administrator\AppData\Local\Temp\ipykernel_36292\3549316113.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  addr.loc[addr.address.str.contains(
C:\Users\Administrator\AppData\Local\Temp\ipykernel_36292\3981848103.py:48: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  is_contain_in_name = lambda x: addr.name.str.contains(x, regex=True, na=False)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_36292\3981848103.py:48: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  is_contain_in_name = lambda x: addr.name.str.contains(x, regex=True, na=False)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_36292\3981848103.py:48: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actuall

,address,name,isContract,isWhale,network,serviceType,sub1Service,walletPurpose,sub1Wallet
0,0x983873529f95132bd1812a3b52c98fb271d2f679,ascendex,False,False,eth,CeFi,exc,NaN,NaN
1,0x19184ab45c40c2920b0e0e31413b9434abd243ed,binance,False,False,eth,CeFi,exc,NaN,NaN
2,0x25681ab599b4e2ceea31f8b498052c53fc2d74db,binance,False,False,eth,CeFi,exc,NaN,NaN
3,0x2859e4544c4bb03966803b044a93563bd2d0dd4d,binance,False,False,eth,CeFi,exc,NaN,NaN
4,0x2bc4eb013ddee29d37920938b96d353171289b7c,binance,False,False,eth,CeFi,exc,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1327,tg8pryf8uqbpudl4mw5f8i11dma4kwuubr,whitebit,False,False,trx,DeFi,NaN,NaN,NaN
1328,tsm3tmmgfnj83q8fnpcsgsk2vzympcfvbc,whitebit,False,False,trx,DeFi,NaN,NaN,NaN
1329,twbpglwqw2ebqyllw1djntdt2zq9yjw1jj,whitebit,False,False,trx,DeFi,NaN,NaN,NaN
1330,tvxtashzddt12sep4fhjhjjbc5t2nlcyjk,xtcom,False,False,trx,DeFi,NaN,NaN,NaN


In [6]:
addr.name.value_counts().sort_index().index

Index(['0x', '1inch', '96ach', 'aave', 'alphapo', 'arbitrum', 'ascendex',
       'aura_finance', 'axelar', 'balancer',
       ...
       'unknownexchange7', 'unknownexchange8', 'unknownexchange9', 'upbit',
       'whitebit', 'wintermute', 'wormhole', 'xtcom', 'xy_finance',
       'yearnfinance'],
      dtype='object', name='name', length=141)

In [8]:
ad = pd.concat([pd.read_csv(f'addr/addresses_data_all_{v}.csv', usecols=[
    'address', 'chain', 'contract', 'isUserAddress', 'arkhamEntity_type', 'arkhamEntity_name', 'arkhamLabel_name', 'arkhamEntity_website'
]).drop_duplicates() for v in ['eth', 'trx']])
ad.address = ad.address.str.lower()

ad.arkhamEntity_type.unique()

array([nan, 'nft-marketplace', 'dex', 'bridge', 'misc', 'dex-aggregator',
       'individual', 'fund', 'cex', 'lending-centralized',
       'fund-decentralized', 'derivatives', 'custodian', 'gaming',
       'lending-decentralized', 'yield', 'government',
       'real-world-assets', 'cdp', 'blockchain-infra', 'dao',
       'smart-contract-platform', 'gambling', 'nft', 'miner-validator',
       'insurance', 'crosschain-interoperability', 'charity',
       'stablecoin', 'blockchain-scaling', 'mev', 'meme', 'staking',
       'hacker', 'prediction-market', 'liquid-staking', 'ai', 'otc',
       'oracle', 'social'], dtype=object)

In [9]:


result = addr.merge(
    ad, on='address', how='outer',indicator=True
)
is_contain_in_type = lambda x: result.arkhamEntity_type.str.contains(x, na=False)
is_contain_in_name = lambda x: result.arkhamLabel_name.str.contains(x, na=False)


result.loc[result.address.str[:2]=='0x', 'network'] = 'eth'
result.loc[result.address.str[0]=='T', 'network'] = 'trx'


result.loc[result.contract==True, 'isContract'] = True # Token?
result.loc[result.arkhamLabel_name.str.contains(r'\([A-Z]+\)', regex=True, na=False), 'isContract'] = True
### Service Type
result.loc[is_contain_in_name('MEV'), 'name'] = 'MEV'
result.loc[is_contain_in_name('MEV'), 'serviceType'] = 'Other'
result.loc[is_contain_in_name('MEV'), 'sub1Service'] = 'MEV'
result.loc[is_contain_in_name('_User'), 'name'] = 'User'
result.loc[is_contain_in_name('_User'), 'serviceType'] = 'Other'
result.loc[is_contain_in_name('_User'), 'sub1Service'] = 'etc'
result.loc[result.isUserAddress==True, 'name'] = 'User'
result.loc[result.isUserAddress==True, 'serviceType'] = 'Other'
result.loc[result.isUserAddress==True, 'sub1Service'] = 'etc'
result.loc[is_contain_in_type('individual'), 'name'] = 'User'
result.loc[is_contain_in_type('individual'), 'serviceType'] = 'Other'
result.loc[is_contain_in_type('individual'), 'sub1Service'] = 'etc'



result.loc[is_contain_in_type('cex'), 'sub1Service'] = 'exc'
result.loc[is_contain_in_type('cex'), 'serviceType'] = 'CeFi'

result.loc[is_contain_in_type('yield'), 'sub1Service'] = 'yield'
result.loc[is_contain_in_type('yield'), 'serviceType'] = 'DeFi'

result.loc[is_contain_in_type('fund'), 'sub1Service'] = 'fund'
result.loc[is_contain_in_type('fund'), 'serviceType'] = 'CeFi'
result.loc[is_contain_in_type('fund-decentralized'), 'serviceType'] = 'DeFi'

result.loc[is_contain_in_type('lending'), 'sub1Service'] = 'lend'
result.loc[is_contain_in_type('lending'), 'serviceType'] = 'CeFi'
result.loc[is_contain_in_type('lending-decentralized'), 'serviceType'] = 'DeFi'


result.loc[is_contain_in_type('custodian'), 'sub1Service'] = 'custody'

result.loc[is_contain_in_type('dex'), 'sub1Service'] = 'exc'
result.loc[is_contain_in_type('dex'), 'serviceType'] = 'DeFi'
result.loc[is_contain_in_type('dex-aggregator'), 'sub1Service'] = 'agg'


result.loc[result.name.isna(), 'name'] = result.loc[result.name.isna(), 'arkhamEntity_name']




import numpy as np


purpose_choices = ['Staking', 'Vault', 'Infra', 'Gov', 'Other']
purpose_conditions = [
    result.arkhamLabel_name.str.contains(v, na=False, regex=True)
    for v in [
        r'(?:Staking)', r'(?:Wallet|Deposit|Pool|USD)',
        r'(?:Mint|Valid|Multisig|Miner)',  r'(?:Voter|Delegat)',
        r'(?:Proxy|Router|Resolver|Market Maker)'
    ]
]

sub_choices = ['cold', 'hot',  'liqPool', 'deposit', 
               'mint', 'multisig', 'validator',
               'proxy', 'router', 'resolver', 'marketMaker']
sub_conditions = [
    result.arkhamLabel_name.str.contains(pattern, na=False, regex=True)
    for pattern in [
        r'(?:Cold Wallet)', r'(?:Hot Wallet)', r'(?:Pool|USD)', r'(?:Deposit)',
        r'(?:Mint)', r'(?:Multisig)', r'(?:Validat|Miner)',
        r'(?:Proxy)', r'(?:Router)', r'(?:Resolver)', r'(?:Market Maker)',
    ]
]

result['walletPurpose'] = np.select(purpose_conditions, purpose_choices, default='')
result['sub1Wallet'] = np.select(sub_conditions, sub_choices, default='')
result.loc[~result.walletPurpose.isna(), 'name'] = result.loc[~result.walletPurpose.isna(), 'name'].str.replace(' wallet','')
print(result._merge.value_counts())

_merge
left_only     27479449
right_only      445361
both             65888
Name: count, dtype: int64


In [11]:
result

,address,name,isContract,isWhale,network,serviceType,sub1Service,walletPurpose,sub1Wallet
0,0x0000000000000000000000000000000000000000,trash,False,False,eth,CeFi,exc,,
1,0x0000000000000000000000000000000000000003,trash,False,False,eth,CeFi,exc,,
2,0x0000000000000000000000000000000000000005,trash,False,False,eth,CeFi,exc,,
3,0x0000000000000000000000000000000000000006,trash,False,False,eth,CeFi,exc,,
4,0x0000000000000000000000000000000000000008,trash,False,False,eth,CeFi,exc,,
...,...,...,...,...,...,...,...,...,...
27990693,tzjzzhvmlgrfbrugrx9p1ph31bdyscms9k,whitebit,False,False,trx,CeFi,exc,,
27990694,tzjzzv6yyeazcmqmekqnoej1vuvbuagwtv,unknownexchange3,False,False,trx,CeFi,exc,,
27990695,tzjzzx14z43uketmhiu8g51ksfrmzbdbzg,changenow,False,False,trx,CeFi,exc,,
27990696,u6pj8dtqupfnfmwhbgfulq4u4egjdiyykjvesynxq2w,gateio,False,False,NaN,CeFi,exc,,


In [ ]:
# result.loc[result.name.str.contains('unknown', na=False)]

In [29]:
result[result.isContract==True][['name', 'isContract', 'isWhale', 'network', 'serviceType',
       'sub1Service', 'walletPurpose', 'sub1Wallet']].value_counts(dropna=False).to_frame()

count
name                          isContract isWhale network serviceType sub1Service walletPurpose sub1Wallet       
NaN                           True       NaN     eth     NaN         NaN         Vault         deposit     10564
                                                 NaN     NaN         NaN         Vault         deposit      6562
                                                 eth     NaN         NaN                                    2279
                                                                                 Other         proxy        2131
Uniswap                       True       NaN     eth     DeFi        exc         Vault         liqPool       782
...                                                                                                          ...
Multibit                      True       NaN     eth     NaN         NaN                                       1
Movement Network              True       NaN     eth     NaN         NaN                                       1
Morpho                        True       NaN     eth     DeFi        lend        Other         proxy           1
Bitget Wallet (Prev. Bitkeep) True       NaN     eth     NaN         NaN                                       1
OpenEden                      True       NaN     eth     NaN         NaN                                       1

[708 rows x 1 columns]

In [10]:
result = result[[
    'address', 'name', 'isContract', 'isWhale', 'network', 'serviceType',
       'sub1Service', 'walletPurpose', 'sub1Wallet',
]].reset_index(drop=True)
print(result.shape)
result.to_csv('addr_info.csv')
result[[
    'serviceType', 'sub1Service', 'walletPurpose', 'sub1Wallet'
]].value_counts(dropna=False).to_frame().sort_index()

(27990698, 9)


count
serviceType sub1Service walletPurpose sub1Wallet          
CeFi        exc                                   27380938
                        Other         proxy              9
                                      router             1
                        Vault                            5
                                      cold              18
...                                                    ...
NaN         NaN         Vault                          227
                                      deposit       265832
                                      hot               19
                                      liqPool          124
                                      proxy              4

[92 rows x 1 columns]

In [3]:
import pandas as pd
import os

result = pd.DataFrame(columns=['name','count','year','month'])

# 20y 10, 11, 12  missing
# 22y 11, 12 missing
for f in os.listdir('meta'):
    try:
        result = pd.concat([result, pd.read_csv(f'meta/{f}')])
    except Exception as e:
        print(f'Error reading {f}: {e}')
        continue
    
t = result.groupby('name').agg({
    'count': 'sum',
    'year': 'count'
})

t.to_parquet('meta/addr_eth')
t

Error reading addr_2016_2023: 'utf-8' codec can't decode bytes in position 7-8: invalid continuation byte


,count,year
name,,
0x,21399710,98
0x0000000000000000000000000000000000000001,189165,99
0x0000000000000000000000000000000000000002,9412,88
0x0000000000000000000000000000000000000004,191,54
0x0000000000000000000000000000000000000007,176,67
...,...,...
zilliqa,3,1
zipmex,4582,26
zkfair,18585,16


In [5]:
t.sort_values('count', ascending=False)

# Validator

In [21]:
result[(result.isContract==True) & (~result.serviceType.isna())]

,address,name,isContract,isWhale,network,serviceType,sub1Service,walletPurpose,sub1Wallet
7,0x000000000000006f6502b7f2bbac8c30a3f67e9a,MEV,True,NaN,eth,Other,MEV,,
10,0x0000000000000eb4ec62758aae93400b3e5f7f18,MEV,True,NaN,eth,Other,MEV,,
12,0x0000000000005117dd3a72e64a705198753fdd54,MEV,True,NaN,eth,Other,MEV,,
13,0x0000000000007f150bd6f54c40a34d7c3d5e9f56,MEV,True,NaN,eth,Other,MEV,,
15,0x00000000000123685885532dcb685c442dc83126,MEV,True,NaN,eth,Other,MEV,,
...,...,...,...,...,...,...,...,...,...
211523,0xfefb05eb5abc0c378120b93ff37cec8af4872a18,DefinitiveFi,True,NaN,eth,DeFi,agg,Vault,liqPool
211959,0xff9127f3d47eadc134a147a9f3ae996444895348,Enzyme Finance,True,NaN,eth,DeFi,fund,Other,proxy
211990,0xff9b21c3bfa4bce9b20b55fed56d102ced48b0f6,Symbiosis Finance,True,NaN,eth,DeFi,exc,,
212119,0xffc9efd66926e3023a4cbd20145ca48f52dff2d9,Uniswap,True,NaN,eth,DeFi,exc,Vault,liqPool


In [22]:
ad[~ad.arkhamLabel_name.str.contains('_User', na=False)].dropna(subset=['arkhamLabel_name'])[[
    'address', 'arkhamEntity_name', 'arkhamEntity_type', 'arkhamLabel_name'
]].drop_duplicates(subset=['arkhamEntity_name', 'arkhamEntity_type', 'arkhamLabel_name']).sort_values('arkhamLabel_name')[25:75]

,address,arkhamEntity_name,arkhamEntity_type,arkhamLabel_name
125642,0x98f90bfc702ec548d21b5a566a4df6853e2890d4,Uniswap,dex,AKRO-USDC
204537,0xf3ad3821a0036fd9caec0a4cf5d31515fad03f00,Balancer,dex,AKRO/USDC 70/30 #4
107771,0x8302efa4e838fd15cf09a39f4ca5bff2a3a78443,Balancer,dex,ALBT/OCEAN/USDC 33.33/33.33/33.33
12475,0x0d88b55317516b84e78fbba7cde3f910d5686901,Balancer,dex,ALBT/USDC/ETH 50/25/25
66846,0x5083b16da538c5022744526122243cf3bddb3bf2,Uniswap,dex,ALI-USDC
64066,0x4a14347083b80e5216ca31350a2d21702ac3650d,Tokenlon,dex,AMMWrapperWithPath
191303,0xd3466427f7ee5d0d9cd68b36df677e51bda26321,Balancer,dex,AMP/USDC 80/20
138149,0xb07fe2f407f971125d4eb1977f8acee8846c7324,Uniswap,dex,APE-USDC 2
138990,0xb27c7b131cf4915bec6c4bc1ce2f33f9ee434b9f,SushiSwap,dex,APE-USDT
4210,0x04a3570a08c72fc0e78ed8b18fd3078585443b4c,Uniswap,dex,API3-USDC


In [11]:
import pandas as pd

summary = pd.read_parquet('meta/addr_eth')
summary # 3min 30sec

,count,year
name,,
0x,21399710,98
0x0000000000000000000000000000000000000001,189165,99
0x0000000000000000000000000000000000000002,9412,88
0x0000000000000000000000000000000000000004,191,54
0x0000000000000000000000000000000000000007,176,67
...,...,...
zilliqa,3,1
zipmex,4582,26
zkfair,18585,16


In [15]:
summary = summary.reset_index()
summary.loc[summary['name'].str.contains('00000000000000000000000000000000000'), 'name'] = 'trash'

In [16]:
summary = summary.drop_duplicates(subset=['name'])
summary

,name,count,year
0,0x,21399710,98
1,trash,189165,99
25137,0x0000000000000000000000000000000000100000,1,1
25138,0x0000000000000000000000000000000000100100,2,1
25139,0x0000000000000000000000000000000000100139,1,1
...,...,...,...
171361015,zilliqa,3,1
171361016,zipmex,4582,26
171361017,zkfair,18585,16
171361018,zkspace,37,15


In [6]:
summary.value_counts('count')

count
1            45238583
2            62096691
3            13287758
4            11389004
5             5260277
               ...   
47627431            1
48980091            1
77224922            1
274300677           1
285664672           1
Name: count, Length: 26816, dtype: int64

In [21]:
summary[summary['count']==1].drop_duplicates().shape[0] / summary.shape[0]

0.2639915375465778

In [22]:
summary[summary['count']==1].drop_duplicates().to_parquet('rare_addr/eth_once.parquet')

In [10]:
summary[summary['count']>2].sort_values('count').tail(1000)

,count,year
name,,
0x1f4ef1f8441caac34f58fb0cba813dd2b09fec63,189283,10
0xab3f9bf1d81ddb224a2014e98b238638824bcf20,189382,61
0x1112956589a2bea1b038732db4ea6b0c416ef130,189619,15
0x00000000008c4fb1c916e0c88fd4cc402d935e7d,189758,28
0x1ffec7119e315b15852557f654ae0052f76e6ae1,189916,15
...,...,...
1inch,47627431,69
coinbase,48980091,78
binance,77224922,95
